# ECM2423 - Coursework Exercise
### Deadline: 17th March 2022 12:00
### Lecturer: Dr. Ayah Helal (a.helal@exeter.ac.uk)

### Question 1: Implement a heuristic search algorithm: A* and
#### Question 1.1: Describe how you would frame the 8-puzzle problem as a search problem

The 8-puzzle problem can be looked at as a search problem, as each possible state of the puzzle can be represented as nodes in a graph, with the arcs representing legal moves between states. The start state is a randomly picked arrangement of tiles. The goal state is the desired final order of tiles. The cost of transition from one state to another is 1. Valid operations are moves where a number adjacent to the blank space moves into the blank space (blank space is swapped with an adjacent tile).

Framing the problem as a search, we are trying to find the path from the start state to the goal state with the lowest cost possible. In game terms, we must order the tiles as instructed with the least number of moves possible.


#### Question 1.2: Solve the 8-puzzle problem using A*

1. In this question you should first briefly outline the A* algorithm

The A* algorithm is a heuristic search algorithm which goes towards the node that appears to be closest to the goal, while also including the cost of reaching said node.

The cheapest path to a goal state through a given node N is equal to the cost of reaching N plus the estimated cost of reaching the goal node from N

Let's say we have start node **S**, current node **C**, and goal node **G**. **P** is a list containing nodes accessed in order (Current path).

f(N) is a function estimating the cheapest path to G passing through N.
g(N) is the cost of reaching N from C.
h(N) is the estimated cost of reaching G from N

`f(N) = g(N) + h(N)`

    1. Start at state S, which is now C. Add C to P.
    2. Calculate f(N) for each adjacent node.
    3. Move to the node with the lowest value of f(N). C now equals N. Add C to the end of P.
    4. If C is the same as G, you have found path P and you can terminate the algorithm.
    5. If not, return to step 2.



2. Describe **two** admissible heuristic functions for the 8-puzzle problem and explain why they are admissible. Make sure you also explain why you chose these two heursitic functions in particular amongst all the possible ones.

**Heuristic 1:** Number of tiles out of place.

This heuristic means that h(N) is equal to the number of tiles out of place in state N. This heuristic is admissible, as it never overestimates; the minimum number of moves to reach a goal state from N is equal to the number of tiles out of place, as each misplaced tile must be moved at least once to reach the goal state. This heuristic was chosen as it's one of the simplest to understand and implement.

**Heuristic 2:** Sum of spaces between tiles and their final states. (Manhattan distance)

This heuristic means that h(N) is the sum of the spaces between all tiles and their required final states; If tile A has 2 slots between it and its final state, tile B has 3, and tile C has 1: h(ABC) = 6. This heuristic is admissible as the number of spaces between each tile is equal to the minimum number of moves to get tiles to their final positions. If there is 3 spaces between A and its final state, the minimum number of moves, and therefore cost to get A to its final state is 3. This heuristic was chosen as it's still fairly simple to understand, however, having more information than heuristic 1 should make it faster.

3. Then, you should implement **two** versions of the A* algorithm in Python to solve the 8-puzzle using the two heuristic functions you have chosen. You can either implement the two versions in the same Python script, letting the user select which one to use before running the code, or you can have two different scripts if you prefer. To test that it works, you can use the start and goal state of Figure 1 (however, note that this may take a few minutes to run depending on your computer, implementation, and choice of heuristic functions), or you can specify your own initial and goal state. If you specify your own initial and goal state, select states which are **at least** five moves apart from each other and **write** these states in your report.



### Heuristic 1: Number of tiles out of place:

In [1]:
from queue import PriorityQueue

class Node:
    def __init__(self, state, depth=0, h=0, parent=None):
        self.state = state
        self.depth = depth
        self.h = h
        self.parent = parent
        self.children = []
    
    # Better string printing
    def __str__(self):
        return ''.join(map(str, ("[",self.state[0][0],",",self.state[0][1],",",self.state[0][2],"]\n[",self.state[1][0],",",self.state[1][1],",",self.state[1][2],"]\n[",self.state[2][0],",",self.state[2][1],",",self.state[2][2],"]")))
    
    # Less than comparison support (for Priorityqueue)
    def __lt__(self, other):
        return self.h < other.h
    
    # Equality support
    def __eq__(self, other):
        return self.state == other.state

    # Hash support (For set)
    def __hash__(self):
        return hash((tuple(self.state[0]),tuple(self.state[1]),tuple(self.state[2])))
    
    # Generates children (Surrounding nodes)
    def gen_children(self, goal_state):
        for move in self.get_legal_moves():
            h = misplaced_heuristic(move, self.depth, goal_state.state)
            c = Node(move, self.depth+1, h, self)
            self.children.append(c)
    
    # Returns a list of legal moves for the current state
    def get_legal_moves(self):
        state = self.state
        
        # Will return an array of legal states
        legal_states = []

        # Getting empty spot
        for row_index in range (3):

                for column_index in range (3):

                    if state[row_index][column_index] == 0:
                        row = row_index
                        column = column_index
                        empty_spot = (row_index, column_index)

        # If the empty tile has a non-empty tile above it
        if row != 0:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row-1][column]
            new_state[row-1][column] = 0
            legal_states.append(new_state)

        # If the empty tile has a non-empty tile below it
        if row != 2:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row+1][column]
            new_state[row+1][column] = 0
            legal_states.append(new_state)

        # If the empty tile has a non-empty tile to the left of it
        if column != 0:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row][column-1]
            new_state[row][column-1] = 0
            legal_states.append(new_state)

        # If the empty tile has a non-empty tile to the right of it
        if column != 2:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row][column+1]
            new_state[row][column+1] = 0
            legal_states.append(new_state)

        return legal_states

# H1 for this problem
def misplaced_heuristic(current_state, current_depth, goal_state):
        out_of_place_tiles = 0
        
        # Will look at all tiles and determine how many are out of place
        for row_index in range (3):
        
            for column_index in range (3):
            
                current_tile = current_state[row_index][column_index]
                goal_tile = goal_state[row_index][column_index]
                
                # Checking if current tile is out of place
                if current_tile != goal_tile and current_tile != 0:
                    out_of_place_tiles += 1
                    
        return out_of_place_tiles + current_depth


def h1_astar_search(current_state, goal_state):
    """
        This function works by keeping track of a set of
        visited states (to ensure we don't go to the same state twice)
        and a priorityqueue, that determines priority by whatever
        node has the lowest heuristic value.
        
        When visiting states, we add them to the visited set,
        and add their children to the priorityqueue with their
        corresponding heuristic value. We then go through this queue,
        checking which states have the lowest cost, and then eventually
        making it to the final state.
    """
    
    visited = set()        
    queue = PriorityQueue()

    queue.put(current_state)

    # Main search code.
    while current_state != goal_state:
        
        current_state = queue.get()

        # If current state hasn't been visited
        if current_state not in visited:
            # Visit the state
            visited.add(current_state)
            
            # Generate children (legal moves)
            current_state.gen_children(goal_state)
            
            # Add children to priorityqueue (ordered by ascending h value)
            for child in current_state.children:
                queue.put(child)


    # This code is just to display the data
    transitions = []
    while current_state is not None:
        transitions.append(current_state)
        current_state = current_state.parent
    
    # This code is just to display the data
    print("State transitions:")
    total_cost = len(transitions) - 1
    while transitions:
        print(transitions.pop())
        if transitions:
            print()
            print("   |")
            print("   V")
        print()
    print("Total cost:",total_cost)
    print("Total states visited:",len(visited))
        
# The start state given by the specification
given_start = Node([[7, 2, 4],
                    [5, 0, 6],
                    [8, 3, 1]])

# The goal state given by the specification
given_goal  = Node([[0, 1, 2],
                    [3, 4, 5],
                    [6, 7, 8]])

h1_astar_search(given_start, given_goal)

State transitions:
[7,2,4]
[5,0,6]
[8,3,1]

   |
   V

[7,2,4]
[0,5,6]
[8,3,1]

   |
   V

[0,2,4]
[7,5,6]
[8,3,1]

   |
   V

[2,0,4]
[7,5,6]
[8,3,1]

   |
   V

[2,5,4]
[7,0,6]
[8,3,1]

   |
   V

[2,5,4]
[7,6,0]
[8,3,1]

   |
   V

[2,5,4]
[7,6,1]
[8,3,0]

   |
   V

[2,5,4]
[7,6,1]
[8,0,3]

   |
   V

[2,5,4]
[7,6,1]
[0,8,3]

   |
   V

[2,5,4]
[0,6,1]
[7,8,3]

   |
   V

[2,5,4]
[6,0,1]
[7,8,3]

   |
   V

[2,5,4]
[6,1,0]
[7,8,3]

   |
   V

[2,5,4]
[6,1,3]
[7,8,0]

   |
   V

[2,5,4]
[6,1,3]
[7,0,8]

   |
   V

[2,5,4]
[6,1,3]
[0,7,8]

   |
   V

[2,5,4]
[0,1,3]
[6,7,8]

   |
   V

[2,5,4]
[1,0,3]
[6,7,8]

   |
   V

[2,5,4]
[1,3,0]
[6,7,8]

   |
   V

[2,5,0]
[1,3,4]
[6,7,8]

   |
   V

[2,0,5]
[1,3,4]
[6,7,8]

   |
   V

[0,2,5]
[1,3,4]
[6,7,8]

   |
   V

[1,2,5]
[0,3,4]
[6,7,8]

   |
   V

[1,2,5]
[3,0,4]
[6,7,8]

   |
   V

[1,2,5]
[3,4,0]
[6,7,8]

   |
   V

[1,2,0]
[3,4,5]
[6,7,8]

   |
   V

[1,0,2]
[3,4,5]
[6,7,8]

   |
   V

[0,1,2]
[3,4,5]
[6,7,8]

Total cost: 26
Total

### Heuristic 2: Sum of spaces between tiles and their final states. (Manhattan distance)


In [2]:
from queue import PriorityQueue

class Node:
    def __init__(self, state, depth=0, h=0, parent=None):
        self.state = state
        self.depth = depth
        self.h = h
        self.parent = parent
        self.children = []
    
    # Better string printing
    def __str__(self):
        return ''.join(map(str, ("[",self.state[0][0],",",self.state[0][1],",",self.state[0][2],"]\n[",self.state[1][0],",",self.state[1][1],",",self.state[1][2],"]\n[",self.state[2][0],",",self.state[2][1],",",self.state[2][2],"]")))
    
    # Less than comparison support (for Priorityqueue)
    def __lt__(self, other):
        return self.h < other.h
    
    # Equality support
    def __eq__(self, other):
        return self.state == other.state

    # Hash support (For set)
    def __hash__(self):
        return hash((tuple(self.state[0]),tuple(self.state[1]),tuple(self.state[2])))
    
    # Generates children (Surrounding nodes)
    def gen_children(self, goal_state):
        for move in self.get_legal_moves():
            h = manhattan_heuristic(move, self.depth, goal_state.state)
            c = Node(move, self.depth+1, h, self)
            self.children.append(c)
    
    # Returns a list of legal moves for the current state
    def get_legal_moves(self):
        state = self.state
        
        # Will return an array of legal states
        legal_states = []

        # Getting empty spot
        for row_index in range (3):

                for column_index in range (3):

                    if state[row_index][column_index] == 0:
                        row = row_index
                        column = column_index
                        empty_spot = (row_index, column_index)

        # If the empty tile has a non-empty tile above it
        if row != 0:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row-1][column]
            new_state[row-1][column] = 0
            legal_states.append(new_state)

        # If the empty tile has a non-empty tile below it
        if row != 2:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row+1][column]
            new_state[row+1][column] = 0
            legal_states.append(new_state)

        # If the empty tile has a non-empty tile to the left of it
        if column != 0:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row][column-1]
            new_state[row][column-1] = 0
            legal_states.append(new_state)

        # If the empty tile has a non-empty tile to the right of it
        if column != 2:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row][column+1]
            new_state[row][column+1] = 0
            legal_states.append(new_state)

        return legal_states

# H2 for this problem
def manhattan_heuristic(current_state, current_depth, goal_state):
        sum = 0
        
        for row_index in range (3):
        
            for column_index in range (3):
            
                # looks at each tile
                current_tile = current_state[row_index][column_index]
                
                foundtile = -1
                
                if current_tile != 0:
                    # If tile isn't 0, then find desired location
                    for desired_row in range (3):
                        for desired_column in range (3):
                            goal_tile = goal_state[desired_row][desired_column]
                            if current_tile == goal_tile:
                                # Exit loop
                                i = 3
                                j = 3
                                # This gets the manhattan distance from tile to desired location
                                sum += abs(row_index - desired_row) + abs(column_index - desired_column)
        
        # print(sum)
        
        return sum + current_depth


def h2_astar_search(current_state, goal_state):
    """
        This function works by keeping track of a set of
        visited states (to ensure we don't go to the same state twice)
        and a priorityqueue, that determines priority by whatever
        node has the lowest heuristic value.
        
        When visiting states, we add them to the visited set,
        and add their children to the priorityqueue with their
        corresponding heuristic value. We then go through this queue,
        checking which states have the lowest cost, and then eventually
        making it to the final state.
    """
    
    visited = set()        
    queue = PriorityQueue()

    queue.put(current_state)

    # Main search code.
    while current_state != goal_state:
        
        current_state = queue.get()

        # If current state hasn't been visited
        if current_state not in visited:
            # Visit the state
            visited.add(current_state)
            
            # Generate children (legal moves)
            current_state.gen_children(goal_state)
            
            # Add children to priorityqueue (ordered by ascending h value)
            for child in current_state.children:
                queue.put(child)


    # This code is just to display the data
    transitions = []
    while current_state is not None:
        transitions.append(current_state)
        current_state = current_state.parent
    
    # This code is just to display the data
    print("State transitions:")
    total_cost = len(transitions) - 1
    while transitions:
        print(transitions.pop())
        if transitions:
            print()
            print("   |")
            print("   V")
        print()
    print("Total cost:",total_cost)
    print("Total states visited:",len(visited))
    

# The start state given by the specification
given_start = Node([[7, 2, 4],
                    [5, 0, 6],
                    [8, 3, 1]])

# The goal state given by the specification
given_goal  = Node([[0, 1, 2],
                    [3, 4, 5],
                    [6, 7, 8]])

h2_astar_search(given_start, given_goal)

State transitions:
[7,2,4]
[5,0,6]
[8,3,1]

   |
   V

[7,2,4]
[0,5,6]
[8,3,1]

   |
   V

[0,2,4]
[7,5,6]
[8,3,1]

   |
   V

[2,0,4]
[7,5,6]
[8,3,1]

   |
   V

[2,5,4]
[7,0,6]
[8,3,1]

   |
   V

[2,5,4]
[7,6,0]
[8,3,1]

   |
   V

[2,5,4]
[7,6,1]
[8,3,0]

   |
   V

[2,5,4]
[7,6,1]
[8,0,3]

   |
   V

[2,5,4]
[7,6,1]
[0,8,3]

   |
   V

[2,5,4]
[0,6,1]
[7,8,3]

   |
   V

[2,5,4]
[6,0,1]
[7,8,3]

   |
   V

[2,5,4]
[6,1,0]
[7,8,3]

   |
   V

[2,5,4]
[6,1,3]
[7,8,0]

   |
   V

[2,5,4]
[6,1,3]
[7,0,8]

   |
   V

[2,5,4]
[6,1,3]
[0,7,8]

   |
   V

[2,5,4]
[0,1,3]
[6,7,8]

   |
   V

[2,5,4]
[1,0,3]
[6,7,8]

   |
   V

[2,5,4]
[1,3,0]
[6,7,8]

   |
   V

[2,5,0]
[1,3,4]
[6,7,8]

   |
   V

[2,0,5]
[1,3,4]
[6,7,8]

   |
   V

[0,2,5]
[1,3,4]
[6,7,8]

   |
   V

[1,2,5]
[0,3,4]
[6,7,8]

   |
   V

[1,2,5]
[3,0,4]
[6,7,8]

   |
   V

[1,2,5]
[3,4,0]
[6,7,8]

   |
   V

[1,2,0]
[3,4,5]
[6,7,8]

   |
   V

[1,0,2]
[3,4,5]
[6,7,8]

   |
   V

[0,1,2]
[3,4,5]
[6,7,8]

Total cost: 26
Total

4. Briefly discuss and compare the results given by A* when using the two different heuristic functions in question 1.2

In both scenarios, the algorithm gave the optimal result for the inputs and goal states given. Both have the same number of moves (26) and follow the exact same steps to reach the goal state. The primary difference between the two heuristic functions was the efficiency and speed of their execution. The misplaced tiles heuristic visited a grand total of **34038** nodes in the graph, compared to the manhattan distance heuristic's total of **2823**, which is a difference of more than 10 times. This means that the space complexity and also time complexity of the misplaced tiles heuristic was much higher, as not only was the overall graph significantly bigger, but the analysis of all those nodes increased the time taken. This is likely due to the fact that the manhattan heuristic provides more information about the desirability of certain nodes; the misplaced tiles heuristic will return the same heuristic for large swathes of tiles with very different desirabilities in practice, meaning overall it results in a much longer calculation.

#### Question 1.3: General solution of the 8-puzzle using A*

In [1]:
# from queue import PriorityQueue

class Node:
    def __init__(self, state, depth=0, h=0, parent=None):
        self.state = state
        self.depth = depth
        self.h = h
        self.parent = parent
        self.children = []
    
    # Better string printing
    def __str__(self):
        return ''.join(map(str, ("[",self.state[0][0],",",self.state[0][1],",",self.state[0][2],"]\n[",self.state[1][0],",",self.state[1][1],",",self.state[1][2],"]\n[",self.state[2][0],",",self.state[2][1],",",self.state[2][2],"]")))
    
    # Less than comparison support (for Priorityqueue)
    def __lt__(self, other):
        return self.h < other.h
    
    # Equality support
    def __eq__(self, other):
        return self.state == other.state

    # Hash support (For set)
    def __hash__(self):
        return hash((tuple(self.state[0]),tuple(self.state[1]),tuple(self.state[2])))
    
    # Generates children (Surrounding nodes)
    def gen_children(self, goal_state):
        for move in self.get_legal_moves():
            h = manhattan_heuristic(move, self.depth, goal_state.state)
            c = Node(move, self.depth+1, h, self)
            self.children.append(c)
    
    # Returns a list of legal moves for the current state
    def get_legal_moves(self):
        state = self.state
        
        # Will return an array of legal states
        legal_states = []

        # Getting empty spot
        for row_index in range (3):

                for column_index in range (3):

                    if state[row_index][column_index] == 0:
                        row = row_index
                        column = column_index
                        empty_spot = (row_index, column_index)

        # If the empty tile has a non-empty tile above it
        if row != 0:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row-1][column]
            new_state[row-1][column] = 0
            legal_states.append(new_state)

        # If the empty tile has a non-empty tile below it
        if row != 2:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row+1][column]
            new_state[row+1][column] = 0
            legal_states.append(new_state)

        # If the empty tile has a non-empty tile to the left of it
        if column != 0:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row][column-1]
            new_state[row][column-1] = 0
            legal_states.append(new_state)

        # If the empty tile has a non-empty tile to the right of it
        if column != 2:
            # Then move that tile into here
            new_state = [v[:] for v in state]
            new_state[row][column] = new_state[row][column+1]
            new_state[row][column+1] = 0
            legal_states.append(new_state)

        return legal_states

# H2 for this problem
def manhattan_heuristic(current_state, current_depth, goal_state):
        sum = 0
        
        for row_index in range (3):
        
            for column_index in range (3):
            
                # looks at each tile
                current_tile = current_state[row_index][column_index]
                
                foundtile = -1
                
                if current_tile != 0:
                    # If tile isn't 0, then find desired location
                    for desired_row in range (3):
                        for desired_column in range (3):
                            goal_tile = goal_state[desired_row][desired_column]
                            if current_tile == goal_tile:
                                # Exit loop
                                i = 3
                                j = 3
                                # This gets the manhattan distance from tile to desired location
                                sum += abs(row_index - desired_row) + abs(column_index - desired_column)
        
        # print(sum)
        
        return sum + current_depth


def generic_astar_search(current_state, goal_state):
    """
        This function works by keeping track of a set of
        visited states (to ensure we don't go to the same state twice)
        and a priorityqueue, that determines priority by whatever
        node has the lowest heuristic value.
        
        When visiting states, we add them to the visited set,
        and add their children to the priorityqueue with their
        corresponding heuristic value. We then go through this queue,
        checking which states have the lowest cost, and then eventually
        making it to the final state.
    """
    
    visited = set()        
    queue = PriorityQueue()

    queue.put(current_state)

    # Main search code.
    while current_state != goal_state:
        
        current_state = queue.get()

        # If current state hasn't been visited
        if current_state not in visited:
            # Visit the state
            visited.add(current_state)
            
            # Generate children (legal moves)
            current_state.gen_children(goal_state)
            
            # Add children to priorityqueue (ordered by ascending h value)
            for child in current_state.children:
                queue.put(child)


    # This code is just to display the data
    transitions = []
    while current_state is not None:
        transitions.append(current_state)
        current_state = current_state.parent
    
    # This code is just to display the data
    print("State transitions:")
    total_cost = len(transitions) - 1
    while transitions:
        print(transitions.pop())
        if transitions:
            print()
            print("   |")
            print("   V")
        print()
    print("Total cost:",total_cost)
    print("Total states visited:",len(visited))


    
print("You will be asked to input the desired values for the 8-puzzle.")
print("They must be input from left to right, top to bottom.")
print("The input must be integers from 0-8, each appearing once in the grid.")
print("Variance from this will lead to errors in the program.")
print()
print("You must now input the start state.")
# The start state given by the user
start = Node([[int(input("Top left tile: ")), int(input("Top middle tile: ")), int(input("Top right tile: "))],
              [int(input("Middle left tile: ")), int(input("Center tile: ")), int(input("Middle right tile: "))],
              [int(input("Bottom left tile: ")), int(input("Bottom middle tile: ")), int(input("Bottom left tile: "))]])

print("You must now input the goal state.")
# The goal state given by the u62
goal  = Node([[int(input("Top left tile: ")), int(input("Top middle tile: ")), int(input("Top right tile: "))],
              [int(input("Middle left tile: ")), int(input("Center tile: ")), int(input("Middle right tile: "))],
              [int(input("Bottom left tile: ")), int(input("Bottom middle tile: ")), int(input("Bottom left tile: "))]])

generic_astar_search(start, goal)

You will be asked to input the desired values for the 8-puzzle.
They must be input from left to right, top to bottom.
The input must be integers from 0-8, each appearing once in the grid.
Variance from this will lead to errors in the program.

You must now input the start state.


KeyboardInterrupt: Interrupted by user

This will solve a large number of 8-puzzle configurations, however there is some configurations which it does not work for.

When having starting tiles

[1, 2, 3]

[4, 6, 5]

[7, 8, 0]

and goal tiles

[1, 2, 3]

[4, 5, 6]

[7, 8, 0]

There is no solution to the puzzle. There is no way to swap the position of the 6 and 5 tiles while adhering to the rules of the game.


## Question 2: Evolutionary algorithm

#### Question 2.1: Design and implement the Sudoko problem using Evolutionary algorithm

1. First, design your evolutionary algorithm addressing the following points in your design process
    
    a) Choose an appropriate solution space and solution representation
    
        The solution space will be a sudoku grid with all values filled in, where each number between 1 and 9 appears exactly once in each vertical slice and once in each horizontal slice of the grid. Beyond that, within every 3x3 subsecction, each number must also appear exactly once. The solutions will be represented as a 2d array, with 9 rows and 9 columns. The grid numbers already input in the program will remain immutable.
    
    b) Define an appropriate fitness function
    
        The fitness function will calculate the number of repeated tiles in rows, columns, and subsections. Fitness will correspond to the percentage of tiles which are repeated accross the whole puzzle.
    
    c) Define a crossover operator for the chosen representation
    
        The crossover operator will select a random of column, row, or subsection from one representation and swap it with the corresponding tiles in another representation.
    
    e) Decide how to initialize the population
    
        The population will be initialized by filling in every 0 in the submitted grid with a randomly chosen number between 1 and 9. This will be done as many times as the population size specified.
    
    f) Decide selection and replacement methods
    
        The selection method will be directly proportional to the fitness function for each individual; the worst 10% in fitness will be discarded, and replaced with crossovers of the top 20%.
    
    g) Choose an appropriate termination criterion
    
        The termination criterion will be that the sudoku grid has no repeating numbers in columns, rows, or subsections.
    
you should first briefly outline how you are representing the Sudoko

2. Then you should implement the evolutionary algorithm in Python to solve the Sudoku problem. You will need to run experiments for the three Sudoku grids provided on the ELE page, for population sizes 10, 100, 1000, 10000. Each experiment needs to be ran 5 times (Each with a different random seed) and average performance across runs considered. In total these amount to 60 runs.



In [1]:
import random
from tokenize import Number

from MySQLdb import NUMBER

### EVOLUTIONARY ALGORITHM ###


def evolve(starting_grid):
    population = create_pop(starting_grid)
    fitness_population = evaluate_pop(population)

    for gen in range(NUMBER_GENERATION):
        mating_pool = select_pop(population, fitness_population)

        offspring_population = crossover_pop(mating_pool)

        population = mutate_pop(offspring_population)

        # Making sure it adheres to input
        population = fix_pop(population, starting_grid)

        fitness_population = evaluate_pop(population)
        best_ind, best_fit = best_pop(population, fitness_population)

        print("Run: "+str(gen)+" fit: "+str(best_fit))
        print(grid_to_string(best_ind))
        if(best_fit == 0):
            gen = NUMBER_GENERATION + 2

### POPULATION-LEVEL OPERATORS ###


def create_pop(starting_grid):
    return [create_ind(starting_grid) for _ in range(POPULATION_SIZE)]


def evaluate_pop(population):
    return [evaluate_ind(individual) for individual in population]


def select_pop(population, fitness_population):
    sorted_population = sorted(
        zip(population, fitness_population), key=lambda ind_fit: ind_fit[1])
    return [individual for individual, fitness in sorted_population[:int(POPULATION_SIZE * TRUNCATION_RATE)]]


def crossover_pop(population):
    return [crossover_ind(random.choice(population), random.choice(population)) for _ in range(POPULATION_SIZE)]


def mutate_pop(population):
    popcopy = [v[:] for v in population]
    return [mutate_ind(individual) for individual in popcopy]


def fix_pop(population, starting_grid):
    return [fix_ind(individual, starting_grid) for individual in population]


def best_pop(population, fitness_population):
    return sorted(zip(population, fitness_population), key=lambda ind_fit: ind_fit[1])[0]

### INDIVIDUAL-LEVEL OPERATORS: REPRESENTATION & PROBLEM SPECIFIC ###


def create_ind(starting_grid):
    individual = [v[:] for v in starting_grid]
    for x in range(9):
        for y in range(9):
            if starting_grid[x][y] == 0:
                individual[x][y] = random.randint(1, 9)
            else:
                individual[x][y] = starting_grid[x][y]

    return individual


def evaluate_ind(individual):
    repetition_sum = 0

    # checking rows
    for x in range(9):
        horz_set = set()
        for y in range(9):
            horz_set.add(individual[x][y])
        repetition_sum += 9 - len(horz_set)

    # checking columns
    for x in range(9):
        vert_set = set()
        for y in range(9):
            vert_set.add(individual[y][x])
        repetition_sum += 9 - len(vert_set)

    # checking squares
    for i in range(3):
        for j in range(3):
            square_set = set()
            for k in range(3):
                for l in range(3):
                    square_set.add(individual[i*3+k][j*3+l])
            repetition_sum += 9 - len(square_set)

    return repetition_sum


def crossover_ind(individual1, individual2):
    swapchoice = random.randint(0, 2)

    if swapchoice == 0:
        swapx = random.randint(0, 8)

        individual1[swapx] = individual2[swapx].copy()

    elif swapchoice == 1:
        swapy = random.randint(0, 8)

        individual1[swapy] = individual2[swapy].copy()

    else:
        swapx = random.randint(0, 2)
        swapy = random.randint(0, 2)

        for k in range(3):
            for l in range(3):
                temp = individual1[swapx*3+k][swapy*3+l]
                individual1[swapx*3+k][swapy*3 +l] = individual2[swapx*3+k][swapy*3+l]

    return random.choice([individual1, individual2])

def mutate_ind(individual):
    if random.random() < MUTATION_RATE:
        for i in range(3):
            swapx = random.randint(0, 8)
            swapy = random.randint(0, 8)
            individual[swapx][swapy] = random.randint(1, 9)

    return individual


def fix_ind(individual, starting_grid):
    for x in range(9):
        for y in range(9):
            if starting_grid[x][y] != 0:
                individual[x][y] = starting_grid[x][y]

    return individual


def grid_to_string(individual):
    stringified = (
        str(individual[0][0]) + str(individual[0][1]) + str(individual[0][2]) + " " + str(individual[0][3]) + str(individual[0][4]) + str(individual[0][5]) + " " + str(individual[0][6])+str(individual[0][7])+str(individual[0][8])+"\n" +
        str(individual[1][0]) + str(individual[1][1]) + str(individual[1][2]) + " " + str(individual[1][3]) + str(individual[1][4]) + str(individual[1][5]) + " " + str(individual[1][6])+str(individual[1][7])+str(individual[1][8])+"\n" +
        str(individual[2][0]) + str(individual[2][1]) + str(individual[2][2]) + " " + str(individual[2][3]) + str(individual[2][4]) + str(individual[2][5]) + " " + str(individual[2][6])+str(individual[2][7])+str(individual[2][8])+"\n" +
        "\n" +
        str(individual[3][0]) + str(individual[3][1]) + str(individual[3][2]) + " " + str(individual[3][3]) + str(individual[3][4]) + str(individual[3][5]) + " " + str(individual[3][6])+str(individual[3][7])+str(individual[3][8])+"\n" +
        str(individual[4][0]) + str(individual[4][1]) + str(individual[4][2]) + " " + str(individual[4][3]) + str(individual[4][4]) + str(individual[4][5]) + " " + str(individual[4][6])+str(individual[4][7])+str(individual[4][8])+"\n" +
        str(individual[5][0]) + str(individual[5][1]) + str(individual[5][2]) + " " + str(individual[5][3]) + str(individual[5][4]) + str(individual[5][5]) + " " + str(individual[5][6])+str(individual[5][7])+str(individual[5][8])+"\n" +
        "\n" +
        str(individual[6][0]) + str(individual[6][1]) + str(individual[6][2]) + " " + str(individual[6][3]) + str(individual[6][4]) + str(individual[6][5]) + " " + str(individual[6][6])+str(individual[6][7])+str(individual[6][8])+"\n" +
        str(individual[7][0]) + str(individual[7][1]) + str(individual[7][2]) + " " + str(individual[7][3]) + str(individual[7][4]) + str(individual[7][5]) + " " + str(individual[7][6])+str(individual[7][7])+str(individual[7][8])+"\n" +
        str(individual[8][0]) + str(individual[8][1]) + str(individual[8][2]) + " " + str(individual[8][3]) + str(individual[8][4]) + str(individual[8][5]) + " " + str(individual[8][6])+str(individual[8][7])+str(individual[8][8])+"\n" +
        "\n"
    )
    return stringified

### PARAMERS VALUES ###


NUMBER_GENERATION = 10000
POPULATION_SIZE = 1000   
TRUNCATION_RATE = 0.2
MUTATION_RATE = 0.1

### EVOLVE! ###

sample_grid = [
    [3, 0, 0, 0, 0, 5, 0, 4, 7],
    [0, 0, 6, 0, 4, 2, 0, 0, 1],
    [0, 0, 0, 0, 0, 7, 8, 9, 0],

    [0, 5, 0, 0, 1, 6, 0, 0, 2],
    [0, 0, 3, 0, 0, 0, 0, 0, 4],
    [8, 1, 0, 0, 0, 0, 7, 0, 0],

    [0, 0, 2, 0, 0, 0, 4, 0, 0],
    [5, 6, 0, 8, 7, 0, 1, 0, 0],
    [0, 0, 0, 3, 0, 0, 6, 0, 0]
]

evolve(sample_grid)


Run: 0 fit: 62
315 855 847
876 342 321
344 517 895

654 716 922
683 591 714
819 481 758

312 951 497
564 877 191
597 373 683


Run: 1 fit: 57
394 685 247
386 342 951
217 367 898

955 416 832
483 534 684
813 491 713

172 661 419
563 872 192
348 328 671


Run: 2 fit: 55
394 835 247
386 942 311
293 167 895

453 416 832
723 533 624
816 529 795

172 951 489
561 875 186
928 371 642


Run: 3 fit: 53
397 895 247
776 942 381
425 167 893

751 316 962
633 834 264
816 254 713

172 678 479
563 874 138
918 396 649


Run: 4 fit: 51
397 165 247
796 942 381
428 167 895

955 916 162
633 528 574
816 299 713

172 261 479
563 874 126
348 371 648


Run: 5 fit: 47
394 835 247
586 942 381
425 167 895

657 116 352
913 528 664
812 499 753

172 735 479
564 874 122
348 361 649


Run: 6 fit: 47
394 465 247
586 942 381
925 147 893

453 716 582
693 528 684
812 954 713

262 735 479
563 879 126
348 381 698


Run: 7 fit: 47
324 875 247
586 842 381
427 167 895

751 416 982
693 528 264
812 954 713

122 938 472
565 874 12

KeyboardInterrupt: 

#### Question 2.2: Analyze the Sudoko problem using Evolutionary algorithm
This question will help guide you to analyize your results based on the following questions.

1. What population size was the best?
2. What do you think is the reason for your findings in question 8.a?
3. Which grid was the easiest and which was the hardest to solve?
4. What do you think might be the reason for your findings in question 8c?
5. What further experiments do you think it may be useful to do and why?